In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

In [3]:
act, act_name = swish, "swish"
print(act_name)

swish


In [4]:
baseMapNum = 32
weight_decay = 1e-4

model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

swish
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
____

In [4]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [6]:
#training
batch_size = 128
epochs=25
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75'+act_name+'.h5')

Epoch 1/75
390/390 [==============================] - 155s - loss: 1.9323 - acc: 0.4188 - val_loss: 1.7221 - val_acc: 0.4713
Epoch 2/75
390/390 [==============================] - 152s - loss: 1.3531 - acc: 0.5720 - val_loss: 1.0408 - val_acc: 0.6731
Epoch 3/75
390/390 [==============================] - 151s - loss: 1.2038 - acc: 0.6329 - val_loss: 1.0127 - val_acc: 0.7021
Epoch 4/75
390/390 [==============================] - 151s - loss: 1.1117 - acc: 0.6674 - val_loss: 0.8938 - val_acc: 0.7361
Epoch 5/75
390/390 [==============================] - 150s - loss: 1.0206 - acc: 0.6943 - val_loss: 0.8497 - val_acc: 0.7431
Epoch 6/75
390/390 [==============================] - 150s - loss: 1.0421 - acc: 0.7026 - val_loss: 0.7800 - val_acc: 0.7689
Epoch 7/75
390/390 [==============================] - 150s - loss: 0.9491 - acc: 0.7197 - val_loss: 0.7157 - val_acc: 0.7880
Epoch 8/75
390/390 [==============================] - 150s - loss: 0.9149 - acc: 0.7342 - val_loss: 0.6919 - val_acc: 0.7979


390/390 [==============================] - 151s - loss: 0.6695 - acc: 0.8327 - val_loss: 0.5424 - val_acc: 0.8671
Epoch 67/75
390/390 [==============================] - 151s - loss: 0.6550 - acc: 0.8350 - val_loss: 0.5180 - val_acc: 0.8712
Epoch 68/75
390/390 [==============================] - 150s - loss: 0.6588 - acc: 0.8339 - val_loss: 0.5597 - val_acc: 0.8631
Epoch 69/75
390/390 [==============================] - 150s - loss: 0.6581 - acc: 0.8379 - val_loss: 0.5244 - val_acc: 0.8684
Epoch 70/75
390/390 [==============================] - 150s - loss: 0.6521 - acc: 0.8370 - val_loss: 0.5658 - val_acc: 0.8605
Epoch 71/75
390/390 [==============================] - 150s - loss: 0.6470 - acc: 0.8365 - val_loss: 0.5517 - val_acc: 0.8634
Epoch 72/75
390/390 [==============================] - 150s - loss: 0.6532 - acc: 0.8369 - val_loss: 0.5375 - val_acc: 0.8651
Epoch 73/75
390/390 [==============================] - 150s - loss: 0.6559 - acc: 0.8363 - val_loss: 0.5371 - val_acc: 0.8720
Epoc

In [7]:
opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 151s - loss: 0.6043 - acc: 0.8498 - val_loss: 0.4878 - val_acc: 0.8841
Epoch 2/25
390/390 [==============================] - 150s - loss: 0.5879 - acc: 0.8557 - val_loss: 0.4923 - val_acc: 0.8858
Epoch 3/25
390/390 [==============================] - 150s - loss: 0.5832 - acc: 0.8567 - val_loss: 0.4915 - val_acc: 0.8853
Epoch 4/25
390/390 [==============================] - 150s - loss: 0.5801 - acc: 0.8596 - val_loss: 0.4907 - val_acc: 0.8898
Epoch 5/25
390/390 [==============================] - 150s - loss: 0.5700 - acc: 0.8584 - val_loss: 0.5023 - val_acc: 0.8825
Epoch 6/25
390/390 [==============================] - 150s - loss: 0.5676 - acc: 0.8604 - val_loss: 0.4728 - val_acc: 0.8867
Epoch 7/25
390/390 [==============================] - 150s - loss: 0.5685 - acc: 0.8608 - val_loss: 0.4883 - val_acc: 0.8866
Epoch 8/25
390/390 [==============================] - 150s - loss: 0.5606 - acc: 0.8600 - val_loss: 0.4678 - val_acc: 0.8853


In [8]:
opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 151s - loss: 0.5076 - acc: 0.8729 - val_loss: 0.4521 - val_acc: 0.8930
Epoch 2/25
390/390 [==============================] - 150s - loss: 0.5077 - acc: 0.8730 - val_loss: 0.4312 - val_acc: 0.8975
Epoch 3/25
390/390 [==============================] - 150s - loss: 0.5029 - acc: 0.8758 - val_loss: 0.4359 - val_acc: 0.8955
Epoch 4/25
390/390 [==============================] - 150s - loss: 0.4987 - acc: 0.8739 - val_loss: 0.4276 - val_acc: 0.8954
Epoch 5/25
390/390 [==============================] - 150s - loss: 0.4902 - acc: 0.8760 - val_loss: 0.4383 - val_acc: 0.8934
Epoch 6/25
390/390 [==============================] - 150s - loss: 0.4975 - acc: 0.8759 - val_loss: 0.4396 - val_acc: 0.8930
Epoch 7/25
390/390 [==============================] - 150s - loss: 0.4911 - acc: 0.8757 - val_loss: 0.4271 - val_acc: 0.8966
Epoch 8/25
390/390 [==============================] - 150s - loss: 0.4867 - acc: 0.8784 - val_loss: 0.4479 - val_acc: 0.8920


In [9]:
#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

 9984/10000 [============================>.] - ETA: 0s
Test result: 89.700 loss: 0.417


In [5]:
baseMapNum = 32
weight_decay = 1e-4

model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.load_weights("models/cifar10_normal_rms_ep125swish.h5")

In [6]:
# Continue for 25 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.00015,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep150'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 155s - loss: 0.4517 - acc: 0.8858 - val_loss: 0.4082 - val_acc: 0.8968
Epoch 2/25
390/390 [==============================] - 152s - loss: 0.4524 - acc: 0.8869 - val_loss: 0.4164 - val_acc: 0.8978
Epoch 3/25
390/390 [==============================] - 151s - loss: 0.4515 - acc: 0.8845 - val_loss: 0.4022 - val_acc: 0.9004
Epoch 4/25
390/390 [==============================] - 152s - loss: 0.4446 - acc: 0.8875 - val_loss: 0.4072 - val_acc: 0.9022
Epoch 5/25
390/390 [==============================] - 152s - loss: 0.4456 - acc: 0.8877 - val_loss: 0.4003 - val_acc: 0.8999
Epoch 6/25
390/390 [==============================] - 152s - loss: 0.4394 - acc: 0.8897 - val_loss: 0.4042 - val_acc: 0.9012
Epoch 7/25
390/390 [==============================] - 152s - loss: 0.4327 - acc: 0.8895 - val_loss: 0.4058 - val_acc: 0.9012
Epoch 8/25
390/390 [==============================] - 152s - loss: 0.4473 - acc: 0.8884 - val_loss: 0.4046 - val_acc: 0.9008


In [7]:
print(his.history)

{'acc': [0.88575721153846154, 0.88689043950606206, 0.88454443373127856, 0.8874719280977833, 0.88767244145011226, 0.88979788256028081, 0.88949711262739961, 0.88847449472557094, 0.88875521334616614, 0.8872313121782498, 0.88999839589348728, 0.88923644528084544, 0.88987808792415934, 0.89057988450433112, 0.88987808794328183, 0.8925649662755184, 0.89158245103650646, 0.88939685591299034, 0.88979788254115844, 0.89007860118087612, 0.89200352903432789, 0.89316650629426031, 0.89019890918844891, 0.89318655754917897, 0.89212383704190079], 'val_loss': [0.40820774867534637, 0.4163949291706085, 0.40224477882385257, 0.40715273625850679, 0.40033424901962278, 0.40424356536865236, 0.40581725521087647, 0.40460287899971009, 0.4070775022983551, 0.40572789745330812, 0.41089151208400726, 0.39454497537612915, 0.39895141930580141, 0.39320371990203856, 0.39213998556137086, 0.3951376909971237, 0.39769589865207672, 0.39982340493202212, 0.38965615992546082, 0.40321386351585387, 0.39857374296188353, 0.396327038788795

In [8]:
# Continue for 25 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000075,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep175'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 151s - loss: 0.4114 - acc: 0.8959 - val_loss: 0.3906 - val_acc: 0.9049
Epoch 2/25
390/390 [==============================] - 150s - loss: 0.4133 - acc: 0.8946 - val_loss: 0.3883 - val_acc: 0.9044
Epoch 3/25
390/390 [==============================] - 150s - loss: 0.4113 - acc: 0.8964 - val_loss: 0.3908 - val_acc: 0.9048
Epoch 4/25
390/390 [==============================] - 150s - loss: 0.4032 - acc: 0.8952 - val_loss: 0.3930 - val_acc: 0.9031
Epoch 5/25
390/390 [==============================] - 150s - loss: 0.4060 - acc: 0.8962 - val_loss: 0.3904 - val_acc: 0.9038
Epoch 6/25
390/390 [==============================] - 150s - loss: 0.4012 - acc: 0.8980 - val_loss: 0.3919 - val_acc: 0.9043
Epoch 7/25
390/390 [==============================] - 150s - loss: 0.4078 - acc: 0.8950 - val_loss: 0.3987 - val_acc: 0.9032
Epoch 8/25
390/390 [==============================] - 150s - loss: 0.4117 - acc: 0.8940 - val_loss: 0.3881 - val_acc: 0.9046


In [9]:
print(his.history)

{'acc': [0.89585336538461535, 0.8945901508051346, 0.89643487327558546, 0.89525184470336716, 0.89623435999974632, 0.89801892847597198, 0.89495107477048597, 0.89402871354482172, 0.8948107154315047, 0.89795877442437944, 0.89637471931960511, 0.89430943216541692, 0.89864051973051007, 0.89699631051677597, 0.89791867176156259, 0.89601379533513137, 0.89585338468386422, 0.89567292272031096, 0.89755774783445619, 0.8976981071351926, 0.89759785049727303, 0.89689605389797877, 0.89807908245107471, 0.89884103306371665, 0.89719682386910493], 'val_loss': [0.39062755880355837, 0.3882546559333801, 0.39080445771217348, 0.39295363612174988, 0.39036970536708832, 0.39192653381824494, 0.39867873466014864, 0.38806623706817628, 0.38853516824245454, 0.39072244238853454, 0.38749761795997617, 0.38951491680145262, 0.38776952061653136, 0.39271495082378388, 0.38921054899692537, 0.3823967484712601, 0.38439375302791595, 0.38494933381080626, 0.38422538604736328, 0.37994479603767395, 0.3864920220375061, 0.380806794810295

In [10]:
# Continue for 5 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000035,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs/5,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep180'+act_name+'.h5')

Epoch 1/5
390/390 [==============================] - 151s - loss: 0.3981 - acc: 0.8982 - val_loss: 0.3786 - val_acc: 0.9057
Epoch 2/5
390/390 [==============================] - 150s - loss: 0.3949 - acc: 0.9000 - val_loss: 0.3826 - val_acc: 0.9049
Epoch 3/5
390/390 [==============================] - 150s - loss: 0.3918 - acc: 0.8996 - val_loss: 0.3828 - val_acc: 0.9046
Epoch 4/5
390/390 [==============================] - 150s - loss: 0.3915 - acc: 0.9004 - val_loss: 0.3818 - val_acc: 0.9042
Epoch 5/5
390/390 [==============================] - 150s - loss: 0.3914 - acc: 0.8997 - val_loss: 0.3815 - val_acc: 0.9054


In [11]:
print(his.history)

{'acc': [0.89819711538461533, 0.90000401022803678, 0.89964308632005285, 0.90036493426987785, 0.89974334295797243], 'val_loss': [0.37863976311683656, 0.3826482698440552, 0.38281142292022707, 0.38175358018875122, 0.38150989553928377], 'loss': [0.398070319990317, 0.39480810997040178, 0.39129868284570585, 0.39154716197940298, 0.39120571573919544], 'val_acc': [0.90569999999999995, 0.90490000000000004, 0.90459999999999996, 0.9042, 0.90539999999999998]}


In [12]:
# Continue for 5 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000017,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs/5,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep185'+act_name+'.h5')

Epoch 1/5
390/390 [==============================] - 151s - loss: 0.3919 - acc: 0.9002 - val_loss: 0.3806 - val_acc: 0.9057
Epoch 2/5
390/390 [==============================] - 150s - loss: 0.3929 - acc: 0.9010 - val_loss: 0.3808 - val_acc: 0.9054
Epoch 3/5
390/390 [==============================] - 14068s - loss: 0.3887 - acc: 0.9009 - val_loss: 0.3818 - val_acc: 0.9053
Epoch 4/5
390/390 [==============================] - 152s - loss: 0.3778 - acc: 0.9024 - val_loss: 0.3796 - val_acc: 0.9054
Epoch 5/5
390/390 [==============================] - 151s - loss: 0.3897 - acc: 0.8999 - val_loss: 0.3781 - val_acc: 0.9064


In [13]:
print(his.history)

{'acc': [0.90020032051282051, 0.90094642286159921, 0.90094642280423187, 0.90239011868475949, 0.90002406159769011], 'val_loss': [0.38060365440845489, 0.38079213035106657, 0.38181369011402128, 0.37961667525768278, 0.37812995083332063], 'loss': [0.39189759615140085, 0.39298692853319978, 0.38876978467395762, 0.37785286450370748, 0.38948825888041699], 'val_acc': [0.90569999999999995, 0.90539999999999998, 0.90529999999999999, 0.90539999999999998, 0.90639999999999998]}


In [14]:
# Continue for 5 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000012,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs/5,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep190'+act_name+'.h5')

Epoch 1/5
390/390 [==============================] - 152s - loss: 0.3867 - acc: 0.9009 - val_loss: 0.3798 - val_acc: 0.9056
Epoch 2/5
390/390 [==============================] - 151s - loss: 0.3844 - acc: 0.9032 - val_loss: 0.3805 - val_acc: 0.9057
Epoch 3/5
390/390 [==============================] - 151s - loss: 0.3890 - acc: 0.9014 - val_loss: 0.3811 - val_acc: 0.9060
Epoch 4/5
390/390 [==============================] - 150s - loss: 0.3892 - acc: 0.8993 - val_loss: 0.3805 - val_acc: 0.9060
Epoch 5/5
390/390 [==============================] - 150s - loss: 0.3823 - acc: 0.9017 - val_loss: 0.3792 - val_acc: 0.9070


In [15]:
print(his.history)

{'acc': [0.90088141025641022, 0.90319217199846302, 0.90146775745909524, 0.89930221370523233, 0.90162816807211765], 'val_loss': [0.37982043609619143, 0.38047481172084807, 0.3811149765968323, 0.3805003794193268, 0.37922024688720701], 'loss': [0.38665701750761422, 0.38440108700826608, 0.38870178570837632, 0.389246430043651, 0.3824876476489163], 'val_acc': [0.90559999999999996, 0.90569999999999995, 0.90600000000000003, 0.90600000000000003, 0.90700000000000003]}
